In [ ]:
suppressPackageStartupMessages(library(scater))
suppressPackageStartupMessages(library(igraph))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages((library(FNN)))
suppressPackageStartupMessages(library(scran))
suppressPackageStartupMessages(library(biomaRt))
suppressPackageStartupMessages(library("xlsx"))
suppressPackageStartupMessages(library("readxl"))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library("ggpubr") )
suppressPackageStartupMessages(library(ggpmisc))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
options(repr.plot.width = 9, repr.plot.height = 7, repr.plot.res = 100)


In [ ]:
markers_root_end_clusters <- findMarkers(assay(new_data4_no_dup,"logcounts")
                                         ,new_data4_no_dup$BinaryCell_w_new_def
                                         , pval.type=c("any", "some", "all")
#                                          ,direction=c("any")
                                         ,direction=c("any","up","down")
                                        )


In [ ]:
markers_root_end_clusters

In [ ]:
# 1) filter by FDR
DF1_up <- lapply(markers_root_end_clusters
      , function(x) apply(x , 2, function(y) subset(x, FDR < 0.05))$FDR          
            )
# 2) subset all columns with logFC, summary.logFC is not important because it reflects 
#               one of the logFC.# columns
DF2_up<-lapply(DF1_up
      , function(x) apply(x , 2, function(y) x[ , grepl( "logFC", names(x))])$FDR
            )
            
# 3) subset logFC > 0.58 , here we use "all" function which means all of
#             the logFC columns must be satisfied!
            
DF3_up<-lapply(DF2_up
      , function(x) x[apply(x , 1, function(y) all(y > 0.58)) ,]
            ) 
# 4) order based on summary.logFC
DF4_up<-lapply(DF3_up
      , function(x) x[order(x$summary.logFC, decreasing = T),]    
            )
# 5) add all of the original columns with the new subset                     
DF5_up<- mapply(function(x, y) list(x[rownames(y),]), DF1_up, DF4_up)
                
                
                
# downregulation===============================================================
                
DF1_down <- lapply(markers_root_end_clusters
, function(x) apply(x , 2, function(y) subset(x, FDR < 0.05))$FDR)
DF2_down<-lapply(DF1_down
, function(x) apply(x , 2, function(y) x[ ,grepl("logFC" ,names(x))])$FDR)
DF3_down<-lapply(DF2_down, function(x) x[apply(x , 1, function(y) all(y < -0.58)) ,]) 
DF4_down<-lapply(DF3_down, function(x) x[order(x$summary.logFC, decreasing = F),])
DF5_down<- mapply(function(x, y) list(x[rownames(y),]), DF1_down, DF4_down)                   

In [ ]:
write.csv(DF5_up[[3]],"progeny/Kira_prog_ETV_root1_vs_others_upreg.csv")
write.csv(DF5_down[[3]],"progeny/Kira_prog_ETV_root1_vs_others_downreg.csv")

write.csv(DF5_up[[1]],"progeny/Kira_prog_ETV_Endpoint1_vs_others_upreg.csv")
write.csv(DF5_down[[1]],"progeny/Kira_prog_ETV_Endpoint1_vs_others_downreg.csv")

write.csv(DF5_up[[2]],"progeny/Kira_prog_ETV_Endpoint2_vs_others_upreg.csv")
write.csv(DF5_down[[2]],"progeny/Kira_prog_ETV_Endpoint2_vs_others_downreg.csv")


In [ ]:
# conditon 2

In [ ]:
# if we don't want to compare vs all of the logFC columns

In [ ]:
# selective

In [ ]:
#  selective 
markers_root_end_clusters2<-markers_root_end_clusters
markers_root_end_clusters2[[2]]<-NULL
not_req<-names(markers_root_end_clusters2)[2]
not_req_col<-paste("logFC",not_req,sep = ".")
not_req_col

# 1) filter by FDR
df1_up <- lapply(markers_root_end_clusters2
      , function(x) apply(x , 2, function(y) subset(x, FDR < 0.05))$FDR          
                        )
                 
# 2) remove unwanted column             
df1_1_up <- lapply(df1_up, function(x) { x[not_req_col] <- NULL; x })
                 
                 
# 3) since summary.logFC might have values from logFC.unwanted column we remove it
#               we also remove Top column by -c(1) but keep other columns
df1_2_up <- lapply(df1_1_up, function(x) { x["summary.logFC"] <- NULL; x }[,-c(1)])
df1_3_up<-lapply(df1_2_up
      , function(x) apply(x , 2, function(y) x[ , grepl( "logFC", names(x))])$FDR
            )
     
                 
# 4) subset logFC > 0.58 , here we use "any" function which means either of
#             the logFC columns must be satisfied! so in the first two columns
#               (p.value and FDR), it certainly is not satisfied so it only needs 
#               to be satisfied in logFC column
            
df2_up<-lapply(df1_3_up
      , function(x) x[apply(x  , 1, function(y) all( y > 0.58)) ,]
                          )                      
      
                      
# 5) add all of the original columns with the new subset                     
df3_up<- mapply(function(x, y) list(x[rownames(y),]), df1_up, df2_up)

# 6) order based on the summary.logFC                         
df4_up<-lapply(df3_up, function(x) x[order(x$summary.logFC, decreasing = T),])
                             
                
#  downregulation==============================================================
                

df1_down <- lapply(markers_root_end_clusters2
        , function(x) apply(x , 2, function(y) subset(x, FDR < 0.05))$FDR)              
df1_1_down <- lapply(df1_down, function(x) { x[not_req_col] <- NULL; x })
df1_2_down <- lapply(df1_1_down, function(x) { x["summary.logFC"] <- NULL; x }[,-c(1)])
df1_3_down<-lapply(df1_2_down
      , function(x) apply(x , 2, function(y) x[ , grepl( "logFC", names(x))])$FDR
            )
                            
df2_down<-lapply(df1_3_down, function(x) x[apply(x  , 1, function(y) all( y < -0.58)) ,]) 
df3_down<- mapply(function(x, y) list(x[rownames(y),]), df1_down, df2_down)
df4_down<-lapply(df3_down, function(x) x[order(x$summary.logFC, decreasing = F),])
                
                

In [ ]:
write.csv(df4_up[[1]],"progeny/Kira_prog_ETV_Endpoint1_vs_Root1_upreg.csv")
write.csv(df4_down[[1]],"progeny/Kira_prog_ETV_Endpoint1_vs_Root1_downreg.csv")


In [ ]:
# conditon 3
# we want only afew clusters vs other subset of clusters not all

In [ ]:
# only poor clusters 1 2 6 7 
markers_root_end_clusters2<-same.markers
not_req<-names(markers_root_end_clusters2)[c(4,9,10,11,12,13,14,15,16,17)]
not_req_col<-paste("logFC",not_req,sep = ".")
not_req_col
# vs All rich clusters 3 5 8
against_req<- c(3,5,8)                         
against_req_col<-paste("logFC",against_req,sep = ".")
against_req_col_2<-gsub("logFC","req_logFC",against_req_col)
against_req_col
against_req_col_2

# =======================
df1_up <- lapply(markers_root_end_clusters2, function(x) { x[not_req_col] <- NULL; x })

df1_1_up <- lapply(df1_up
      , function(x) apply(x , 2, function(y) subset(x, FDR <= 0.05))$FDR)                        

# we rename colnames that need to be compared with                          
library(qdap)                      
df1_2_up <- lapply(df1_1_up, function(x) {
                 names(x)<- mgsub(against_req_col,against_req_col_2,names(x))  
  x
})
                          
df1_3_up<-lapply(df1_2_up
      , function(x) apply(x , 2, function(y) x[ , grepl("req_logFC", names(x))])$FDR)
df2_up<-lapply(df1_3_up
      , function(x) x[apply(x  , 1, function(y) all( y >= 0.58)) ,])                                        
df3_up<- mapply(function(x, y) list(x[rownames(y),]), df1_up, df2_up)                       
df4_up<-lapply(df3_up, function(x) x[order(x$summary.logFC, decreasing = T),])
# #  downregulation==============================================================
df1_down <- lapply(markers_root_end_clusters2, function(x) { x[not_req_col] <- NULL; x })

df1_1_down <- lapply(df1_down
      , function(x) apply(x , 2, function(y) subset(x, FDR <= 0.05))$FDR)                        
                          
df1_2_down <- lapply(df1_1_down, function(x) {
                 names(x)<- mgsub(against_req_col,against_req_col_2,names(x))  
  x
})
                          
df1_3_down<-lapply(df1_2_down
      , function(x) apply(x , 2, function(y) x[ , grepl("req_logFC", names(x))])$FDR)
df2_down<-lapply(df1_3_down
      , function(x) x[apply(x  , 1, function(y) all( y <= -0.58)) ,])                                        
df3_down<- mapply(function(x, y) list(x[rownames(y),]), df1_down, df2_down )                      
df4_down<-lapply(df3_down, function(x) x[order(x$summary.logFC, decreasing = F),])